In [6]:
import joblib
import pandas as pd
from config import load_config
from ml_utils import load_tmap_labels
import os
os.chdir("/datadisk1/ixk5174/project_repo/Telos-test/")

project_config = load_config("project_config/dRNA-NA12878_isoquant_config.pkl")

df_cov = pd.read_csv(project_config.cov_file, sep="\t")
df_label = load_tmap_labels(project_config.tmap_file)
df_tss = pd.read_csv(project_config.tss_labeled_file)
df_tes = pd.read_csv(project_config.tes_labeled_file)

df_tss.drop(columns=["strand"], inplace=True)
df_tes.drop(columns=["strand"], inplace=True)


print(project_config.cov_file)
print(df_cov.head())

print(f"df_tss.shape: {df_tss.shape}")
print(f"df_tes.shape: {df_tes.shape}")
print(f"df_cov.shape: {df_cov.shape}")
print(f"df_label.shape: {df_label.shape}")

df = df_cov.merge(df_tss, left_on=["tss_chrom", "tss_pos"], right_on=["chrom", "position"], how="inner")
df.drop(columns=["chrom", "position"], inplace=True)
print(f"after merging df_tss, df.shape: {df.shape}")
df_dropped = df_cov[df_cov["transcript_id"].isin(df["transcript_id"].unique())]
df_dropped = df_dropped[["transcript_id", "tss_chrom", "tss_pos", "strand"]]
print(df_dropped.head())

df = df.merge(df_tes, left_on=["tes_chrom", "tes_pos"], right_on=["chrom", "position"], how="inner", suffixes=("_tss", "_tes"))
print(f"after merging df_tes, df.shape: {df.shape}")
df = df.merge(df_label , left_on=["transcript_id"], right_on=["transcript_id"], how="inner")
print(f"after merging df_label, df.shape: {df.shape}")

print(f"merged df.shape: {df.shape}")

# dropped rows values
df_dropped = df_cov[~df_cov["transcript_id"].isin(df["transcript_id"].unique())]
df_dropped = df_dropped[["transcript_id", "tss_chrom", "tss_pos", "tes_chrom", "tes_pos", "strand"]]
df_dropped.head()



train_output/dRNA-NA12878_isoquant/data/dRNA-NA12878_isoquant_cov.tsv
            transcript_id strand tss_chrom  tss_pos tes_chrom  tes_pos  \
0   transcript1.chr1.nnic      -      chr1   199855      chr1    14404   
1   transcript3.chr1.nnic      -      chr1   195287      chr1   184921   
2   transcript5.chr1.nnic      -      chr1   195419      chr1    14403   
3   transcript7.chr1.nnic      -      chr1   189024      chr1   184921   
4  transcript11.chr1.nnic      -      chr1    84042      chr1    14406   

   coverage  first_exon_length  last_exon_length  max_exon_length  \
0         0                 19               426              426   
1         0                 25               430              430   
2         0               1663               427             4951   
3         0                234               430              512   
4         0                206               633              633   

   min_exon_length  mean_exon_length  total_exon_length  exon_count  


,transcript_id,tss_chrom,tss_pos,tes_chrom,tes_pos,strand
0,transcript1.chr1.nnic,chr1,199855,chr1,14404,-
1,transcript3.chr1.nnic,chr1,195287,chr1,184921,-
2,transcript5.chr1.nnic,chr1,195419,chr1,14403,-
3,transcript7.chr1.nnic,chr1,189024,chr1,184921,-
4,transcript11.chr1.nnic,chr1,84042,chr1,14406,-


In [8]:
from ml_utils import load_tmap_labels
from config import load_config
import pandas as pd
import os 
import numpy as np
from ml_utils import stratified_split

os.chdir("/datadisk1/ixk5174/project_repo/Telos-test/")
print(os.getcwd())

project_config = load_config("project_config/cDNA-NA12878_isoquant_config.pkl")
pred_df = pd.read_csv(f"{project_config.predictions_output_dir}/randomforest_stage2_predictions_train.csv", sep = "\t")

# find accuracy of model
acc = 0
inc_count = 0
# pred_df['pred_label'] = pred_df['pred_label'].astype(int)
for i in range(len(pred_df)):
    if pred_df.iloc[i]['pred_label'] == pred_df.iloc[i]['label']:
        acc += 1
    else:
        # print(f"pred_df.iloc[i]:  {pred_df.iloc[i]}")
        inc_count += 1
print(f"Accuracy of model: {acc/len(pred_df)}")
print(f"Inc_count: {inc_count}")




/datadisk1/ixk5174/project_repo/Telos-test
Accuracy of model: 0.9345549738219895
Inc_count: 250
